The notebook is still WIP and will not be considered as part of the submissions for task 2. It's purpose is just to illustrate how the csv files in BCG - Covid-19 challenge could be merged together in order to work on the analysis. Everyone should feel free to copy the notebook and modify it as they wish. 

In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
# Get the latest BCG vaccine policy information for each contry
bcg_strain_country_policy = pd.read_csv('/kaggle/input/hackathon/task_2-BCG_strain_per_country-8Nov2020.csv', delimiter=',')
bcg_strain_country_policy.tail()


In [ ]:
# Get the unified BCG Strain spreadsheet
bcg_strain_type = pd.read_csv('/kaggle/input/hackathon/BCG-Strain.csv', delimiter=',')
pd.set_option('display.max_columns', None)
bcg_strain_type.head()

For each country show the BCG Strain Class (Duplicatio Group based on this paper - "[The BCG Strain Pool: Diversity Matters](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4817828/)" by Daria Bottai and Roland Brosch) and "T Cell Epitopes Group 3" of the BCG Strain (based on "[Genome Sequencing and Analysis of BCG Vaccine Strains](https://www.researchgate.net/publication/256102624_Genome_Sequencing_and_Analysis_of_BCG_Vaccine_Strains)" paper by Wem Zhang et al.)[](http://)

In [ ]:
bcg_strain_type_country_policy = pd.merge(bcg_strain_country_policy, bcg_strain_type, left_on='mandatory_bcg_strain_2015-2020',right_on='Internal BCG Strain ID',how='left',suffixes=('_left','_right'))
bcg_strain_type_country_policy[["country_name", "country_code", "mandatory_bcg_strain_2015-2020", "Class", "T Cell Epitopes Group 3"]].tail()

In [ ]:
# Get the COVID-19 mortality statistics
covid_death_cases = pd.read_csv('/kaggle/input/hackathon/task_2-COVID-19-death_cases_per_country_after_fifth_death-till_22_September_2020.csv', delimiter=',')
covid_death_cases.tail()

In [ ]:
# Get country related information which might be useful for the COVID-19 analysis, e.g. income group, median age, 
country_data = pd.read_csv('/kaggle/input/hackathon/BCG_country_data.csv', delimiter=',')
country_data.tail()

In [ ]:
# Get "alpha 2 code" for each country in covid_death_cases
covid_deatch_cases_country = pd.merge(covid_death_cases, country_data, left_on='alpha_3_code',right_on='alpha_3_code',how='left',
        suffixes=('_left','_right'))
covid_deatch_cases_country[["country_name_left", 
                            "alpha_2_code", 
                            "alpha_3_code",                             
                            "deaths_per_million_50_days_after_fifth_death", 
                            "deaths_per_million_100_days_after_fifth_death", 
                            "deaths_per_million_150_days_after_fifth_death"]].tail()

In [ ]:
# Merege contry, covid-19 and bcg policy information
covid_deatch_cases_bcg_strain = pd.merge(covid_deatch_cases_country, bcg_strain_type_country_policy, left_on='alpha_2_code',right_on='country_code',how='left',suffixes=('_left','_right'))
covid_deatch_cases_bcg_strain.tail()

In [ ]:
covid_deatch_cases_bcg_strain[[
    "country_name_left", 
    "country_code", 
    "alpha_3_code", 
    "mandatory_bcg_strain_2015-2020", 
    "Class", "T Cell Epitopes Group 3", 
    "deaths_per_million_50_days_after_fifth_death", 
    "deaths_per_million_100_days_after_fifth_death", 
    "deaths_per_million_150_days_after_fifth_death"]].tail()

In [ ]:
#covid_deatch_cases_bcg_strain[covid_deatch_cases_bcg_strain.mandatory_bcg_strain_2015 == "Unknown_BCG_Strain 	"].tail()
covid_deatch_cases_bcg_strain.columns = covid_deatch_cases_bcg_strain.columns.str.replace('-','_')
covid_deatch_cases_bcg_strain.columns = covid_deatch_cases_bcg_strain.columns.str.replace(' ','_')


In [ ]:
# Remove the records where:
# * we don't know whether BCG vaccine has been mandatory in the last 5 years;
# * deaths_per_million_50_days_after_fifth_death is not null; (TBD)
# * the countries are not High or Upper-middle income group. (TBD)
covid_deatch_cases_bcg_strain_filtered = covid_deatch_cases_bcg_strain[
        covid_deatch_cases_bcg_strain.mandatory_bcg_strain_2015_2020.notnull()]

covid_deatch_cases_bcg_strain_filtered[[
    "country_name_left", 
    "country_code", 
    "income_group_type", 
    "population_per_km2",
    "mandatory_bcg_strain_2015_2020", 
    "Class", "T_Cell_Epitopes_Group_3", 
    "deaths_per_million_50_days_after_fifth_death", 
    "deaths_per_million_100_days_after_fifth_death", 
    "deaths_per_million_150_days_after_fifth_death"]].tail()

In [ ]:
# save the result in a csv file
covid_deatch_cases_bcg_strain_filtered.to_csv("bcg_covid_filtered.csv")